In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX TITAN X (UUID: GPU-220d4b5f-cb36-56a7-bc5f-fc109cfd4569)


In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

2024-06-02 14:56:50.449783: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [1]:
import os

import pandas as pd
import math
import random as rd
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import pickle
import dlib

from deepface import DeepFace
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
from deepface.extendedmodels import Age, Gender, Race, Emotion
#from deepface.modules import verification
from deepface.commons import functions, distance as dst

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,LSTM,\
                            TimeDistributed,Flatten,Dense,Bidirectional,ConvLSTM2D,MaxPooling3D,AveragePooling2D,Lambda,\
                            Activation,BatchNormalization

from tensorflow import keras
tf.test.gpu_device_name()

2024-08-02 20:36:33.462839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [2]:
seed_constant = 27
np.random.seed(seed_constant)
rd.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
DIR = '/var/scratch/mpa326/'

In [4]:
import pickle
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_50m_50f', 'rb')     
df = pickle.load(dbfile)

y_train_50m_50f = df['labels']
gender_train_50m_50f = df['genders']
X_train_50m_50f=df['features']
paths_train_50m_50f = df['video_files_paths']

In [5]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_40m_60f', 'rb')     
df = pickle.load(dbfile)

y_train_40m_60f = df['labels']
gender_train_40m_60f = df['genders']
X_train_40m_60f=df['features']
paths_train_40m_60f = df['video_files_paths']

In [6]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_60m_40f', 'rb')     
df = pickle.load(dbfile)

y_train_60m_40f = df['labels']
gender_train_60m_40f = df['genders']
X_train_60m_40f=df['features']
paths_train_60m_40f = df['video_files_paths']

In [7]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_test_common','rb')
df = pickle.load(dbfile)

y_test = df['labels']
gender_test = df['genders']
X_test=df['features']
paths_test = df['video_files_paths']

In [8]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_50m_50f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_50m_50f=df['features']

In [9]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_40m_60f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_40m_60f=df['features']

In [10]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_60m_40f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_60m_40f=df['features']

In [11]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_test_common', 'rb')     
df = pickle.load(dbfile)
X_test_aud=df['features']

In [12]:
# X_test = np.concatenate((X_val, X_test), axis=0)
# X_aud_test = np.concatenate((X_val_aud, X_test_aud), axis=0)
# y_test = np.concatenate((y_val, y_test), axis=0)
# gender_test = np.concatenate((gender_val, gender_test), axis=0)

In [20]:
# df_train = pd.DataFrame({'paths_train': paths_train})
# df_val = pd.DataFrame({'paths_val': paths_val})
# df_test = pd.DataFrame({'paths_test': paths_test})

# # Export each DataFrame to a separate CSV file
# df_train.to_csv('paths_train.csv', index=False)
# df_val.to_csv('paths_val.csv', index=False)
# df_test.to_csv('paths_test.csv', index=False)

In [22]:
# def create_new_path(old_path):
#     # Extract the filename from the old path
#     filename = os.path.basename(old_path)
#     # Replace the first "02" with "03"
#     new_filename = filename.replace("02", "03", 1)
#     # Change the extension from .mp4 to .png
#     new_filename = os.path.splitext(new_filename)[0] + '.png'
#     # Create the new path
#     new_path = f"/var/scratch/mpa326/ravdess_melspec_new/{new_filename}"
#     return new_path

# df_train['melspec_path'] = df_train['paths_train'].apply(create_new_path)
# df_train.to_csv('paths_train_audio.csv', index=False)
# df_val['melspec_path'] = df_val['paths_val'].apply(create_new_path)
# df_val.to_csv('paths_val_audio.csv', index=False)
# df_test['melspec_path'] = df_test['paths_test'].apply(create_new_path)
# df_test.to_csv('paths_test_audio.csv', index=False)


In [6]:
# file_path = 'video_genders.csv'

# # Save NumPy array to CSV with newline character as delimiter
# np.savetxt(file_path, genders, delimiter='\n', fmt='%s')

In [12]:
from sklearn.utils import shuffle
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=27)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=7)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=42)

In [13]:
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=27)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=7)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=42)

In [14]:
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=27)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=7)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=42)

In [15]:
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=27)
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=7)
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=42)

In [16]:
from tensorflow.keras.models import Model
emotion=Emotion.loadModel()
for layer in emotion.layers:
    print(layer.name)
features_emotion=Model(inputs=emotion.input, outputs=emotion.get_layer('dense_1').output)

conv2d
max_pooling2d
conv2d_1
conv2d_2
average_pooling2d
conv2d_3
conv2d_4
average_pooling2d_1
flatten
dense
dropout
dense_1
dropout_1
dense_2


In [17]:
features_emotion.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 44, 44, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2  (None, 20, 20, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 18, 64)        36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 average_pooling2d (Average  (None, 7, 7, 64)          0         
 Pooling2D)                                                  

In [18]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_50m_50f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_50m_50f=df_
df_=[]

In [19]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_40m_60f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_40m_60f=df_
df_=[]

In [20]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_60m_40f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_60m_40f=df_
df_=[]

In [21]:
df_ = np.ones((288,20,48,48))
for i in range(288):
  for j in range(20):
    img=X_test[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_test=df_
df_=[]

In [22]:
# indices = np.arange(len(X_train_50m_50f))
# np.random.shuffle(indices)
# X_train_50m_50f = X_train_50m_50f[indices]
# X_train_aud_50m_50f = X_train_aud_50m_50f[indices]
# y_train_50m_50f = y_train_50m_50f[indices]
# gender_train_50m_50f = gender_train_50m_50f[indices]

In [23]:
# indices = np.arange(len(X_train_40m_60f))
# np.random.shuffle(indices)
# X_train_40m_60f = X_train_40m_60f[indices]
# X_train_aud_40m_60f = X_train_aud_40m_60f[indices]
# y_train_40m_60f = y_train_40m_60f[indices]
# gender_train_40m_60f = gender_train_40m_60f[indices]

In [25]:
# indices = np.arange(len(X_train_60m_40f))
# np.random.shuffle(indices)
# X_train_60m_40f = X_train_60m_40f[indices]
# X_train_aud_60m_40f = X_train_aud_60m_40f[indices]
# y_train_60m_40f = y_train_60m_40f[indices]
# gender_train_60m_40f = gender_train_60m_40f[indices]

In [28]:
# indices = np.arange(len(X_test))
# np.random.shuffle(indices)

# X_test = X_test[indices]
# X_test_aud = X_test_aud[indices]
# y_test = y_test[indices]
# gender_test = gender_test[indices]

In [22]:
features_emotion.trainable=False

In [30]:
#Spatial transformer network (STN)

In [23]:
class Localization(tf.keras.layers.Layer):
    def __init__(self):
        super(Localization, self).__init__()
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv1 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D()
        self.conv2 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(20, activation='relu')
        self.fc2 = tf.keras.layers.Dense(6, activation=None, bias_initializer=tf.keras.initializers.constant([1.0, 0.0, 0.0, 0.0, 1.0, 0.0]), kernel_initializer='zeros')

    def build(self, input_shape):
        print("Building Localization Network with input shape:", input_shape)

    def compute_output_shape(self, input_shape):
        return [None, 2, 3]

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        theta = self.fc2(x)
        theta = tf.keras.layers.Reshape((2, 3))(theta)
        return theta

In [24]:
class BilinearInterpolation(tf.keras.layers.Layer):
    def __init__(self, height=40, width=40):
        super(BilinearInterpolation, self).__init__()
        self.height = height
        self.width = width

    def compute_output_shape(self, input_shape):
        return [None, self.height, self.width, 1]

    def get_config(self):
        return {
            'height': self.height,
            'width': self.width,
        }
    
    def build(self, input_shape):
        print("Building Bilinear Interpolation Layer with input shape:", input_shape)

    def advance_indexing(self, inputs, x, y):
        '''
        Numpy like advance indexing is not supported in tensorflow, hence, this function is a hack around the same method
        '''        
        shape = tf.shape(inputs)
        batch_size, _, _ = shape[0], shape[1], shape[2]
        
        batch_idx = tf.range(0, batch_size)
        batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
        b = tf.tile(batch_idx, (1, self.height, self.width))
        indices = tf.stack([b, y, x], 3)
        return tf.gather_nd(inputs, indices)

    def call(self, inputs):
        images, theta = inputs
        homogenous_coordinates = self.grid_generator(batch=tf.shape(images)[0])
        return self.interpolate(images, homogenous_coordinates, theta)

    def grid_generator(self, batch):
        x = tf.linspace(-1, 1, self.width)
        y = tf.linspace(-1, 1, self.height)
            
        xx, yy = tf.meshgrid(x, y)
        xx = tf.reshape(xx, (-1,))
        yy = tf.reshape(yy, (-1,))
        homogenous_coordinates = tf.stack([xx, yy, tf.ones_like(xx)])
        homogenous_coordinates = tf.expand_dims(homogenous_coordinates, axis=0)
        homogenous_coordinates = tf.tile(homogenous_coordinates, [batch, 1, 1])
        homogenous_coordinates = tf.cast(homogenous_coordinates, dtype=tf.float32)
        return homogenous_coordinates
    
    def interpolate(self, images, homogenous_coordinates, theta):

        with tf.name_scope("Transformation"):
            transformed = tf.matmul(theta, homogenous_coordinates)
            transformed = tf.transpose(transformed, perm=[0, 2, 1])
            transformed = tf.reshape(transformed, [-1, self.height, self.width, 2])
                
            x_transformed = transformed[:, :, :, 0]
            y_transformed = transformed[:, :, :, 1]
                
            x = ((x_transformed + 1.) * tf.cast(self.width, dtype=tf.float32)) * 0.5
            y = ((y_transformed + 1.) * tf.cast(self.height, dtype=tf.float32)) * 0.5

        with tf.name_scope("VariableCasting"):
            x0 = tf.cast(tf.math.floor(x), dtype=tf.int32)
            x1 = x0 + 1
            y0 = tf.cast(tf.math.floor(y), dtype=tf.int32)
            y1 = y0 + 1

            x0 = tf.clip_by_value(x0, 0, self.width-1)
            x1 = tf.clip_by_value(x1, 0, self.width-1)
            y0 = tf.clip_by_value(y0, 0, self.height-1)
            y1 = tf.clip_by_value(y1, 0, self.height-1)
            x = tf.clip_by_value(x, 0, tf.cast(self.width, dtype=tf.float32)-1.0)
            y = tf.clip_by_value(y, 0, tf.cast(self.height, dtype=tf.float32)-1)

        with tf.name_scope("AdvanceIndexing"):
            Ia = self.advance_indexing(images, x0, y0)
            Ib = self.advance_indexing(images, x0, y1)
            Ic = self.advance_indexing(images, x1, y0)
            Id = self.advance_indexing(images, x1, y1)

        with tf.name_scope("Interpolation"):
            x0 = tf.cast(x0, dtype=tf.float32)
            x1 = tf.cast(x1, dtype=tf.float32)
            y0 = tf.cast(y0, dtype=tf.float32)
            y1 = tf.cast(y1, dtype=tf.float32)
                            
            wa = (x1-x) * (y1-y)
            wb = (x1-x) * (y-y0)
            wc = (x-x0) * (y1-y)
            wd = (x-x0) * (y-y0)

            wa = tf.expand_dims(wa, axis=3)
            wb = tf.expand_dims(wb, axis=3)
            wc = tf.expand_dims(wc, axis=3)
            wd = tf.expand_dims(wd, axis=3)
                        
        return tf.math.add_n([wa*Ia + wb*Ib + wc*Ic + wd*Id])

In [16]:
emotions_en = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]

In [26]:
def create__LRCN_with_STN() :
  image = tf.keras.layers.Input(shape=(20,48,48,1))
  theta = TimeDistributed(Localization())(image)
  x = TimeDistributed(BilinearInterpolation(height=48, width=48))([image, theta])
  features=TimeDistributed(features_emotion)(x)
  lstm=Bidirectional(LSTM(200, activation='tanh',input_shape=(20, 1024),dropout=.3))(features)
  out=Dense(len(emotions_en), activation = 'softmax')(lstm)

  return tf.keras.models.Model(inputs=image, outputs=out)


In [27]:
create__LRCN_with_STN().summary()

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20, 48, 48, 1)]      0         []                            
                                                                                                  
 time_distributed (TimeDist  (None, 20, 2, 3)             43086     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 time_distributed_1 (TimeDi  (None, 20, 48, 48, 1)        0         ['input_1[0][0]',             
 stributed)                        

In [17]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [18]:
from sklearn.model_selection import StratifiedKFold

seed = 7
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [30]:
unique_values, counts = np.unique(gender_train_50m_50f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value female appears 480 times
Value male appears 480 times


In [32]:
unique_values, counts = np.unique(gender_train_40m_60f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")

Value female appears 576 times
Value male appears 384 times


In [34]:
unique_values, counts = np.unique(gender_train_60m_40f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")

Value female appears 384 times
Value male appears 576 times


In [35]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = os.path.abspath(DIR)+'/ravdess_60m40f_video_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(X_train_60m_40f, y_train_60m_40f):
  # with tpu_strategy.scope():
  model=create__LRCN_with_STN() 
  model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
  early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)
  checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
        monitor='val_accuracy', 
        save_best_only=True, mode='max')

  LRCN_model_training_history = model.fit(        x = X_train_60m_40f[train_idx],
                                                  y = y_train_60m_40f[train_idx],
                                                  validation_data=(X_train_60m_40f[val_idx],y_train_60m_40f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 8,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb,early_stopping_callback])

  model.load_weights(save_dir+"model_"+str(fold_var)+".h5")
	
  results = model.evaluate(X_test, y_test)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_ACCURACY.append(results['accuracy'])
  VALIDATION_LOSS.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))
Epoch 1/80
96/96 [==============================] - 26s 211ms/step - loss: 1.5091 - accuracy: 0.4414 - val_loss: 1.3383 - val_accuracy: 0.4792
Epoch 2/80
96/96 [==============================] - 18s 192ms/step - loss: 1.2113 - accuracy: 0.5599 - val_loss: 1.2176 - val_accuracy: 0.5052
Epoch 3/80
96/96 [==============================] - 18s 192ms/step - loss: 1.0401 - accuracy: 0.5977 - val_loss: 1.0755 - val_accuracy: 0.6042
Epoch 4/80
96/96 [==============================] - 18s 192ms/step - loss: 0.9077 - accuracy: 0.6628 - val_loss: 1.1551 - val_accuracy: 0.5625
Epoch 5/80
96/96 [==============================] - 19s 194ms/step - loss: 0.7977 - accuracy: 0.7174 - val_loss: 1.0790 - val_accuracy: 0.6198
Epoch 6/80
96/96 [==============================] - 18s 191ms/step - loss: 0.8750 - accuracy: 0.6771 - val_loss: 1.1670 - val_acc

In [42]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/ravdess_50m50f_video_model/'
final_model = create__LRCN_with_STN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(X_test, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = X_test[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'positive_rate': np.mean(y_pred)
    }

#Additional fairness metrics
positive_rates = [metrics_by_gender[gender]['positive_rate'] for gender in metrics_by_gender]
disparate_impact_ratio = min(positive_rates) / max(positive_rates)

print("Gender bias analysis:", metrics_by_gender)
print("Disparate Impact Ratio:", disparate_impact_ratio)

for gender in metrics_by_gender:
    metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['fpr_value'], metrics_by_gender[gender]['fnr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']

    
# avg_fpr_values = [np.mean(fpr) for fpr in fpr_value]
# avg_fnr_values = [np.mean(fnr) for fnr in fnr_value]

# false_positive_equality = max(avg_fpr_values) - min(avg_fpr_values)
# false_negative_equality = max(avg_fnr_values) - min(avg_fnr_values)

# print("False Positive Equality:", false_positive_equality)
# print("False Negative Equality:", false_negative_equality) 


# for gender in metrics_by_gender:
#     print(f"Gender: {gender}")
#     print(f"  Demographic Parity: {metrics_by_gender[gender]['demographic_parity']}")
#     print(f"  Equalized Odds (FPR, FNR): {metrics_by_gender[gender]['equalized_odds']}")
#     print(f"  Equal Opportunity (TPR): {metrics_by_gender[gender]['equal_opportunity']}")
#     print(f"  Predictive Parity: {metrics_by_gender[gender]['predictive_parity']}")


data = {
    'Metric': [
        'Accuracy', 'Precision', 'Recall', 'F1 Score', 'True Positive Rate', 'False Positive Rate',
        'False Negative Rate', 'Predictive Parity', 'Positive Rate', 'Demographic Parity',
        'Equalized Odds (FPR, FNR)', 'Equal Opportunity (TPR)'
    ],
    'Male': [
        metrics_by_gender['male']['accuracy'], metrics_by_gender['male']['precision'],
        metrics_by_gender['male']['recall'], metrics_by_gender['male']['f1'],
        metrics_by_gender['male']['tpr'], metrics_by_gender['male']['fpr_value'],
        metrics_by_gender['male']['fnr_value'], metrics_by_gender['male']['precision_value'],
        metrics_by_gender['male']['positive_rate'], metrics_by_gender['male']['demographic_parity'],
        metrics_by_gender['male']['equalized_odds'], metrics_by_gender['male']['equal_opportunity']
    ],
    'Female': [
        metrics_by_gender['female']['accuracy'], metrics_by_gender['female']['precision'],
        metrics_by_gender['female']['recall'], metrics_by_gender['female']['f1'],
        metrics_by_gender['female']['tpr'], metrics_by_gender['female']['fpr_value'],
        metrics_by_gender['female']['fnr_value'], metrics_by_gender['female']['precision_value'],
        metrics_by_gender['female']['positive_rate'], metrics_by_gender['female']['demographic_parity'],
        metrics_by_gender['female']['equalized_odds'], metrics_by_gender['female']['equal_opportunity']
    ]
}

# Create DataFrame and display
df = pd.DataFrame(data)
#csv_file_path = 'video_metrics_results_7th_Jul.csv'
df.to_csv(csv_file_path, index=False)


#gender_accuracy[gender] = final_model.evaluate(X_test[gender_indices], y_test[gender_indices])[1]


Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))
9/9 [==============================] - 4s 146ms/step - loss: 7.7533 - accuracy: 0.1319
Test set evaluation: {'loss': 7.753340721130371, 'accuracy': 0.1319444477558136}
5/5 [==============================] - 1s 136ms/step
Gender bias analysis: {'female': {'accuracy': 0.1111111111111111, 'precision': 0.10728945761840498, 'recall': 0.11085526315789473, 'f1': 0.10767020569804298, 'tpr': array([0.1       , 0.        , 0.26315789, 0.15789474, 0.05263158,
       0.05      , 0.10526316, 0.15789474]), 'fpr_value': array([0.08955224, 0.144     , 0.104     , 0.2       , 0.104     ,
       0.09677419, 0.136     , 0.144     ]), 'fnr_value': array([0.9       , 1.        , 0.73684211, 0.84210526, 0.94736842,
       0.95      , 0.89473684, 0.84210526]), 'precision_value': array([0.07692308, 0.        , 0.27777778, 0.10714286, 0.07142857,
       0.0

NameError: name 'csv_file_path' is not defined

In [42]:
unique_values, counts = np.unique(gender_test, return_counts=True)

# Displaying the results
print("Unique values:", unique_values)
print("Counts:", counts)


Unique values: ['female' 'male']
Counts: [107 109]


In [18]:
#Audio

In [19]:
def model_alexnet():
  AlexNet = Sequential()

  #1st Convolutional Layer
  AlexNet.add(Conv2D(filters=96, input_shape=(128,128,3), kernel_size=(11,11), strides=(4,4), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #2nd Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #3rd Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #4th Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #5th Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #Passing it to a Fully Connected layer
  AlexNet.add(Flatten())
  AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(Dropout(0.4))

  # #2nd Fully Connected Layer
  # AlexNet.add(Dense(4096))
  # AlexNet.add(BatchNormalization())
  # AlexNet.add(Activation('relu'))
  # #Add Dropout
  # AlexNet.add(Dropout(0.4))

  #3rd Fully Connected Layer
  AlexNet.add(Dense(1000))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #Output Layer
  AlexNet.add(Dense(8))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('softmax'))

  return AlexNet 

In [20]:
model_alexnet().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 96)        34944     
                                                                 
 batch_normalization (Batch  (None, 32, 32, 96)        384       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 256)       614656    
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 256)       1

In [21]:
base_model = tf.keras.applications.EfficientNetV2B1(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)
# Freeze base model
base_model.trainable = False
def model_feature_extraction():
  inputs = keras.Input(shape=(224, 224, 3))
  x = tf.cast(inputs, tf.float32)
  # x = tf.keras.applications.efficientnet.preprocess_input(x)
  x = base_model(inputs, training=False)
  x = keras.layers.GlobalAveragePooling2D()(x)
  inter = keras.layers.Dense(1024,activation='relu')(x)
  outputs = keras.layers.Dense(8,activation = 'softmax')(inter)
  model = keras.Model(inputs, outputs)
  model.summary()

  return model 

In [22]:
# dbfile = open('data_audio_1D', 'rb')
# df_aud = pickle.load(dbfile)
# labels = df_aud['labels']


# dbfile = open('data_mel_new', 'rb')
# df_audio_1=pickle.load(dbfile)
# df_audio = df_audio_1['features']
# labels = df_audio_1['labels']
# genders_audio = df_audio_1['genders']
# paths = df_audio_1['video_files_paths']

In [46]:
# dbfile = open('audio_train', 'rb')
# df = pickle.load(dbfile)
# X_train_aud = df['features']
# y_train_aud = df['labels']
# gender_train_aud = df['genders']
# paths_train_aud = df['audio_file_paths']

In [47]:
# dbfile = open('audio_val', 'rb')
# df = pickle.load(dbfile)
# X_val_aud = df['features']
# y_val_aud = df['labels']
# gender_val_aud = df['genders']
# paths_val_aud = df['audio_file_paths']

In [48]:
# dbfile = open('audio_test', 'rb')
# df = pickle.load(dbfile)
# X_test_aud = df['features']
# y_test_aud = df['labels']
# gender_test_aud = df['genders']
# paths_test_aud = df['audio_file_paths']

In [49]:
# emotion_to_index = {emotion: idx for idx, emotion in enumerate(emotions_en)}
# y_train_aud = np.array([emotion_to_index[emotion] for emotion in y_train_aud], dtype=int)
# y_val_aud = np.array([emotion_to_index[emotion] for emotion in y_val_aud], dtype=int)
# y_test_aud = np.array([emotion_to_index[emotion] for emotion in y_test_aud], dtype=int)
# y_train_aud

array([3, 5, 4, ..., 3, 6, 4])

In [40]:
# print(type(df_audio))

# #df_audio = np.array(df_audio)

# # Check the type to confirm the conversion
# print(type(df_audio))  # Output should be <class 'numpy.ndarray'>
# print(df_audio.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1440, 128, 128, 3)


In [22]:
# def resize_images(images, new_size=(128, 128)):
#     resized_images = tf.image.resize(images, new_size)
#     return resized_images

# # Resize the dataset
# df_audio_tensor = tf.convert_to_tensor(df_audio)
# df_audio_resized = resize_images(df_audio_tensor, new_size=(128, 128))
# df_audio_resized = df_audio_resized.numpy()

In [23]:
# df_audio_resized.shape

(1440, 128, 128, 3)

In [23]:
# labels = np.array(labels)
# print(labels.shape)

(1440,)


In [24]:
# #genders = np.array(genders_audio)
# print(genders.shape)

(1440,)


In [25]:
# combined_labels = [(e, g) for e, g in zip(labels, genders)]
# X_train, X_temp, y_train, y_temp, gender_train, gender_temp = train_test_split(
#     df_audio, labels, genders, test_size=0.3, stratify=combined_labels, random_state=42
# )

# combined_temp_labels = [(e, g) for e, g in zip(y_temp, gender_temp)]

# # Split the temporary set into validation and test sets
# X_val, X_test, y_val, y_test, gender_val, gender_test = train_test_split(
#     X_temp, y_temp, gender_temp, test_size=0.5, stratify=combined_temp_labels, random_state=42
# )

# print("Train set shape:", X_train.shape)
# print("Validation set shape:", X_val.shape)
# print("Test set shape:", X_test.shape)

# print("Label Train set shape:", y_train.shape)
# print("Label Validation set shape:", y_val.shape)
# print("Label Test set shape:", y_test.shape)

# print("GenderTrain set shape:", gender_train)
# print("Gender Validation set shape:", gender_val.shape)
# print("Gender Test set shape:", gender_test.shape)

Train set shape: (1008, 128, 128, 3)
Validation set shape: (216, 128, 128, 3)
Test set shape: (216, 128, 128, 3)
Label Train set shape: (1008,)
Label Validation set shape: (216,)
Label Test set shape: (216,)
GenderTrain set shape: ['female' 'male' 'female' ... 'female' 'male' 'male']
Gender Validation set shape: (216,)
Gender Test set shape: (216,)


In [ ]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH= []

save_dir = os.path.abspath(DIR)+'/ravdess_60m40f_audio_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(X_train_aud_60m_40f, y_train_60m_40f):
  # with tpu_strategy.scope():
  model=model_alexnet() 
  model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
  early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)
  checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
        monitor='val_accuracy', 
        save_best_only=True, mode='max')

  LRCN_model_training_history = model.fit(        x = X_train_aud_60m_40f[train_idx],
                                                  y = y_train_60m_40f[train_idx],
                                                  validation_data=(X_train_aud_60m_40f[val_idx], y_train_60m_40f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 8,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb,early_stopping_callback])

  model.load_weights(save_dir+"model_"+str(fold_var)+".h5")
	
  results = model.evaluate(X_test_aud, y_test)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
  VALIDATION_LOSS_SPEECH.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1

2024-07-29 21:05:06.297423: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.
2024-07-29 21:05:06.846655: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.


Epoch 1/80
96/96 [==============================] - 822s 8s/step - loss: 2.1121 - accuracy: 0.1979 - val_loss: 43.6466 - val_accuracy: 0.1406
Epoch 2/80
96/96 [==============================] - 1114s 11s/step - loss: 1.8630 - accuracy: 0.2734 - val_loss: 2.1062 - val_accuracy: 0.2552
Epoch 3/80
96/96 [==============================] - 1037s 11s/step - loss: 1.7733 - accuracy: 0.3529 - val_loss: 2.9946 - val_accuracy: 0.2917
Epoch 4/80
96/96 [==============================] - 988s 10s/step - loss: 1.7091 - accuracy: 0.3542 - val_loss: 5.9470 - val_accuracy: 0.1823
Epoch 5/80
96/96 [==============================] - 991s 10s/step - loss: 1.5800 - accuracy: 0.4245 - val_loss: 3.6494 - val_accuracy: 0.1771
Epoch 6/80
96/96 [==============================] - 967s 10s/step - loss: 1.5318 - accuracy: 0.4531 - val_loss: 2.9489 - val_accuracy: 0.2500
Epoch 7/80
96/96 [==============================] - 1047s 11s/step - loss: 1.4463 - accuracy: 0.4792 - val_loss: 1.6246 - val_accuracy: 0.3802
Epo

In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/saved_model_feature_extraction_audio/'
final_model = model_alexnet()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(X_test_aud, y_test_aud)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = X_test[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr': fpr,
        'fnr': fnr,
        'precision_value': precision_value,
        'positive_rate': np.mean(y_pred)
    }

#Additional fairness metrics
positive_rates = [metrics_by_gender[gender]['positive_rate'] for gender in metrics_by_gender]
disparate_impact_ratio = min(positive_rates) / max(positive_rates)

print("Gender bias analysis:", metrics_by_gender)
print("Disparate Impact Ratio:", disparate_impact_ratio)

for gender in metrics_by_gender:
    metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['fpr'], metrics_by_gender[gender]['fnr'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']

# avg_fpr_values = [np.mean(fpr) for fpr in fpr_values]
# avg_fnr_values = [np.mean(fnr) for fnr in fnr_values]

# false_positive_equality = max(avg_fpr_values) - min(avg_fpr_values)
# false_negative_equality = max(avg_fnr_values) - min(avg_fnr_values)

# print("False Positive Equality:", false_positive_equality)
# print("False Negative Equality:", false_negative_equality) 


# for gender in metrics_by_gender:
#     print(f"Gender: {gender}")
#     print(f"  Demographic Parity: {metrics_by_gender[gender]['demographic_parity']}")
#     print(f"  Equalized Odds (FPR, FNR): {metrics_by_gender[gender]['equalized_odds']}")
#     print(f"  Equal Opportunity (TPR): {metrics_by_gender[gender]['equal_opportunity']}")
#     print(f"  Predictive Parity: {metrics_by_gender[gender]['predictive_parity']}")


data = {
    'Metric': [
        'Accuracy', 'Precision', 'Recall', 'F1 Score', 'True Positive Rate', 'False Positive Rate',
        'False Negative Rate', 'Predictive Parity', 'Positive Rate', 'Demographic Parity',
        'Equalized Odds (FPR, FNR)', 'Equal Opportunity (TPR)'
    ],
    'Male': [
        metrics_by_gender['male']['accuracy'], metrics_by_gender['male']['precision'],
        metrics_by_gender['male']['recall'], metrics_by_gender['male']['f1'],
        metrics_by_gender['male']['tpr'], metrics_by_gender['male']['fpr'],
        metrics_by_gender['male']['fnr'], metrics_by_gender['male']['precision_value'],
        metrics_by_gender['male']['positive_rate'], metrics_by_gender['male']['demographic_parity'],
        metrics_by_gender['male']['equalized_odds'], metrics_by_gender['male']['equal_opportunity']
    ],
    'Female': [
        metrics_by_gender['female']['accuracy'], metrics_by_gender['female']['precision'],
        metrics_by_gender['female']['recall'], metrics_by_gender['female']['f1'],
        metrics_by_gender['female']['tpr'], metrics_by_gender['female']['fpr'],
        metrics_by_gender['female']['fnr'], metrics_by_gender['female']['precision_value'],
        metrics_by_gender['female']['positive_rate'], metrics_by_gender['female']['demographic_parity'],
        metrics_by_gender['female']['equalized_odds'], metrics_by_gender['female']['equal_opportunity']
    ]
}

# Create DataFrame and display
df = pd.DataFrame(data)
csv_file_path = 'audio_metrics_results_7th_Jul.csv'
df.to_csv(csv_file_path, index=False)

7/7 [==============================] - 9s 1s/step - loss: 1.1040 - accuracy: 0.6343
Test set evaluation: {'loss': 1.1039944887161255, 'accuracy': 0.6342592835426331}
4/4 [==============================] - 2s 570ms/step
Gender bias analysis: {'female': {'accuracy': 0.6448598130841121, 'precision': 0.6562150961643238, 'recall': 0.6327380952380952, 'f1': 0.6217810342810343, 'tpr': array([0.42857143, 0.78571429, 0.78571429, 0.28571429, 0.78571429,
       0.4       , 0.73333333, 0.85714286]), 'fpr': array([0.02      , 0.05376344, 0.12903226, 0.02150538, 0.03225806,
       0.0326087 , 0.04347826, 0.07526882]), 'fnr': array([0.57142857, 0.21428571, 0.21428571, 0.71428571, 0.21428571,
       0.6       , 0.26666667, 0.14285714]), 'precision_value': array([0.6       , 0.6875    , 0.47826087, 0.66666667, 0.78571429,
       0.66666667, 0.73333333, 0.63157895]), 'positive_rate': 3.7757009345794392}, 'male': {'accuracy': 0.6238532110091743, 'precision': 0.5869543200674099, 'recall': 0.58035714285714

In [35]:
unique_values, counts = np.unique(gender_test, return_counts=True)

# Displaying the results
print("Unique values:", unique_values)
print("Counts:", counts)

Unique values: ['female' 'male']
Counts: [107 109]


In [ ]:
#Multi-model - Late fusion

In [46]:
indice_fold = 1

In [47]:
from tensorflow.keras.models import load_model

save_dir = os.path.abspath(DIR)+'/saved_model_feature_extraction_audio/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(indice_fold)+".h5")

In [48]:
save_dir = os.path.abspath(DIR)+'/saved_models_lrcn_stn/'

model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(indice_fold)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [49]:
model_video.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 20, 48, 48, 1)]      0         []                            
                                                                                                  
 time_distributed_33 (TimeD  (None, 20, 2, 3)             43086     ['input_13[0][0]']            
 istributed)                                                                                      
                                                                                                  
 time_distributed_34 (TimeD  (None, 20, 48, 48, 1)        0         ['input_13[0][0]',            
 istributed)                                                         'time_distributed_33[0][0]'] 
                                                                                           

In [50]:
print(df)

                       Metric  \
0                    Accuracy   
1                   Precision   
2                      Recall   
3                    F1 Score   
4          True Positive Rate   
5         False Positive Rate   
6         False Negative Rate   
7           Predictive Parity   
8               Positive Rate   
9          Demographic Parity   
10  Equalized Odds (FPR, FNR)   
11    Equal Opportunity (TPR)   

                                                 Male  \
0                                             0.53211   
1                                            0.571701   
2                                            0.514286   
3                                            0.519221   
4   [0.2857142857142857, 0.6666666666666666, 0.4, ...   
5   [0.0392156862745098, 0.0851063829787234, 0.095...   
6   [0.7142857142857143, 0.3333333333333333, 0.6, ...   
7   [0.3333333333333333, 0.5555555555555556, 0.4, ...   
8                                            3.486239   


In [53]:
from sklearn.model_selection import StratifiedKFold
seed=7
kfold=StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
indices=[]
for train_index, test_index in kfold.split(df, labels):
    indices.append([train_index, test_index])


In [54]:
print(labels)

[0 0 0 ... 7 7 7]


In [56]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('bidirectional_11').output)

In [57]:
features_video=features_video_model.predict(df)

45/45 [==============================] - 10s 142ms/step


In [58]:
print(features_video)

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  2.6046636e-37 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -3.3500925e-08  2.1837667e-33 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 ...
 [ 0.0000000e+00 -0.0000000e+00  7.6159418e-01 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  7.1184909e-01 ...  0.0000000e+00
  -0.0000000e+00 -0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  7.6538086e-01 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


In [59]:
features_video.shape

(1440, 400)

In [61]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('dropout_7').output)

In [62]:
features_audio=features_audio_model.predict(df_audio)

45/45 [==============================] - 60s 1s/step


In [63]:
features_audio.shape

(1440, 1000)

In [64]:
data=np.concatenate((features_video,features_audio),axis=1)

In [65]:
print(data.shape)
print(data)

(1440, 1400)
[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  2.6046636e-37 ...  0.0000000e+00
   1.4337306e+00  0.0000000e+00]
 [ 0.0000000e+00 -3.3500925e-08  2.1837667e-33 ...  0.0000000e+00
   3.8423204e-01  0.0000000e+00]
 ...
 [ 0.0000000e+00 -0.0000000e+00  7.6159418e-01 ...  2.1384232e+00
   0.0000000e+00  3.3266492e+00]
 [ 0.0000000e+00  0.0000000e+00  7.1184909e-01 ...  2.3069346e+00
   0.0000000e+00  2.3839350e+00]
 [ 0.0000000e+00  0.0000000e+00  7.6538086e-01 ...  8.3070189e-01
   2.4034429e-01  1.1089498e+00]]


In [50]:

#!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.8/660.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.0 MB/s eta 0:00:00


In [66]:
indice_train,indice_test=indices[indice_fold]

In [97]:
combined_labels = [(e, g) for e, g in zip(labels, genders)]
X_train, X_temp, y_train, y_temp, gender_train, gender_temp = train_test_split(
    data, labels, genders, test_size=0.3, stratify=combined_labels, random_state=42
)

combined_temp_labels = [(e, g) for e, g in zip(y_temp, gender_temp)]

# Split the temporary set into validation and test sets
X_val, X_test, y_val, y_test, gender_val, gender_test = train_test_split(
    X_temp, y_temp, gender_temp, test_size=0.5, stratify=combined_temp_labels, random_state=42
)

print("Train set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

print("Label Train set shape:", y_train.shape)
print("Label Validation set shape:", y_val.shape)
print("Label Test set shape:", y_test.shape)

print("GenderTrain set shape:", gender_train)
print("Gender Validation set shape:", gender_val.shape)
print("Gender Test set shape:", gender_test.shape)

Train set shape: (1008, 1400)
Validation set shape: (216, 1400)
Test set shape: (216, 1400)
Label Train set shape: (1008,)
Label Validation set shape: (216,)
Label Test set shape: (216,)
GenderTrain set shape: ['female' 'male' 'female' ... 'female' 'male' 'male']
Gender Validation set shape: (216,)
Gender Test set shape: (216,)


In [52]:
# X_train, y_train = data[indice_train],labels[indice_train]
# X_test, y_test = data[indice_test],labels[indice_test]

In [98]:
X_val.shape

(216, 1400)

In [99]:
import gc

import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

import optuna

In [100]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

In [90]:
from sklearn.decomposition import PCA

lda = PCA(n_components=800)
X_train = lda.fit_transform(X_train)
X_test = lda.transform(X_test)
X_val = lda.transform(X_val)

In [101]:
print(X_train.shape)

(1008, 1400)


In [39]:
def objective(trial):
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    X_combined = np.vstack((X_train, X_val))
    y_combined = np.hstack((y_train, y_val))
    
    # Initialize the logistic regression model with the suggested C value
    clf = linear_model.LogisticRegression(C=logreg_c)
    
    # Fit the model on the combined training and validation data
    clf.fit(X_combined, y_combined)
    preds = clf.predict(X_test)
    score = accuracy_score(y_test, preds)
    gc.collect()
    return score

In [103]:
from sklearn import linear_model
study = optuna.create_study(study_name="tps-feb-2022", direction="maximize")
study.optimize(objective, n_trials=30)

[I 2024-06-15 20:51:45,562] A new study created in memory with name: tps-feb-2022
[I 2024-06-15 20:52:02,496] Trial 0 finished with value: 0.8564814814814815 and parameters: {'logreg_c': 3.582137263067999}. Best is trial 0 with value: 0.8564814814814815.
[I 2024-06-15 20:52:17,784] Trial 1 finished with value: 0.8657407407407407 and parameters: {'logreg_c': 2671763.636383394}. Best is trial 1 with value: 0.8657407407407407.
[I 2024-06-15 20:52:48,265] Trial 2 finished with value: 0.8611111111111112 and parameters: {'logreg_c': 0.003013327616642366}. Best is trial 1 with value: 0.8657407407407407.
[I 2024-06-15 20:53:12,076] Trial 3 finished with value: 0.8611111111111112 and parameters: {'logreg_c': 0.30502941590720467}. Best is trial 1 with value: 0.8657407407407407.
[I 2024-06-15 20:54:20,480] Trial 4 finished with value: 0.8564814814814815 and parameters: {'logreg_c': 3.223338847539848e-07}. Best is trial 1 with value: 0.8657407407407407.
[I 2024-06-15 20:54:36,884] Trial 5 finished

In [104]:
trial = study.best_trial
print("  Best accuracy: ", study.best_value)
print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Best accuracy:  0.9120370370370371
  Value: 0.9120370370370371
  Params: 
    logreg_c: 277.9763062288022


In [107]:
from sklearn.linear_model import LogisticRegression
best_params = study.best_params
logreg_c = best_params["logreg_c"]
X_combined = np.vstack((X_train, X_val))
y_combined = np.hstack((y_train, y_val))
best_clf = LogisticRegression(C=logreg_c)
best_clf.fit(X_combined, y_combined)

LogisticRegression(C=277.9763062288022)

In [108]:
best_preds = best_clf.predict(X_test)

In [110]:
overall_metrics = {
    'accuracy': accuracy_score(y_test, best_preds),
    'precision': precision_score(y_test, best_preds, average='macro'),
    'recall': recall_score(y_test, best_preds, average='macro'),
    'f1_score': f1_score(y_test, best_preds, average='macro')
}

print("Test set evaluation:", overall_metrics)

Test set evaluation: {'accuracy': 0.9120370370370371, 'precision': 0.9084532076719576, 'recall': 0.9083743842364532, 'f1_score': 0.9079476449782163}


In [111]:
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = X_test[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = best_clf.predict(X_gender_test)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr': fpr,
        'fnr': fnr,
        'precision_value': precision_value,
        'positive_rate': np.mean(y_pred)
    }

# Additional fairness metrics
positive_rates = [metrics_by_gender[gender]['positive_rate'] for gender in metrics_by_gender]
disparate_impact_ratio = min(positive_rates) / max(positive_rates)

print("Gender bias analysis:", metrics_by_gender)
print("Disparate Impact Ratio:", disparate_impact_ratio)

for gender in metrics_by_gender:
    metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['fpr'], metrics_by_gender[gender]['fnr'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']

fpr_values = [metrics_by_gender[gender]['fpr'] for gender in metrics_by_gender]
fnr_values = [metrics_by_gender[gender]['fnr'] for gender in metrics_by_gender]

avg_fpr_values = [np.mean(fpr) for fpr in fpr_values]
avg_fnr_values = [np.mean(fnr) for fnr in fnr_values]

false_positive_equality = max(avg_fpr_values) - min(avg_fpr_values)
false_negative_equality = max(avg_fnr_values) - min(avg_fnr_values)

print("False Positive Equality:", false_positive_equality)
print("False Negative Equality:", false_negative_equality)

Gender bias analysis: {'female': {'accuracy': 0.8878504672897196, 'precision': 0.8894551526904468, 'recall': 0.8779761904761905, 'f1': 0.8795235128469973, 'tpr': array([0.71428571, 1.        , 1.        , 0.86666667, 0.8       ,
       0.71428571, 1.        , 0.92857143]), 'fpr': array([0.01      , 0.        , 0.03225806, 0.04347826, 0.01086957,
       0.01075269, 0.01075269, 0.01075269]), 'fnr': array([0.28571429, 0.        , 0.        , 0.13333333, 0.2       ,
       0.28571429, 0.        , 0.07142857]), 'precision_value': array([0.83333333, 1.        , 0.82352941, 0.76470588, 0.92307692,
       0.90909091, 0.93333333, 0.92857143]), 'positive_rate': 3.682242990654206}, 'male': {'accuracy': 0.9357798165137615, 'precision': 0.9348679812834224, 'recall': 0.9380952380952381, 'f1': 0.9325628458258464, 'tpr': array([1.        , 1.        , 1.        , 0.71428571, 1.        ,
       1.        , 0.85714286, 0.93333333]), 'fpr': array([0.00980392, 0.0106383 , 0.        , 0.01052632, 0.0212766

In [112]:
data = {
    'Metric': [
        'Accuracy', 'Precision', 'Recall', 'F1 Score', 'True Positive Rate', 'False Positive Rate',
        'False Negative Rate', 'Predictive Parity', 'Positive Rate', 'Demographic Parity',
        'Equalized Odds (FPR, FNR)', 'Equal Opportunity (TPR)'
    ],
    'Male': [
        metrics_by_gender['male']['accuracy'], metrics_by_gender['male']['precision'],
        metrics_by_gender['male']['recall'], metrics_by_gender['male']['f1'],
        metrics_by_gender['male']['tpr'], metrics_by_gender['male']['fpr'],
        metrics_by_gender['male']['fnr'], metrics_by_gender['male']['precision_value'],
        metrics_by_gender['male']['positive_rate'], metrics_by_gender['male']['demographic_parity'],
        metrics_by_gender['male']['equalized_odds'], metrics_by_gender['male']['equal_opportunity']
    ],
    'Female': [
        metrics_by_gender['female']['accuracy'], metrics_by_gender['female']['precision'],
        metrics_by_gender['female']['recall'], metrics_by_gender['female']['f1'],
        metrics_by_gender['female']['tpr'], metrics_by_gender['female']['fpr'],
        metrics_by_gender['female']['fnr'], metrics_by_gender['female']['precision_value'],
        metrics_by_gender['female']['positive_rate'], metrics_by_gender['female']['demographic_parity'],
        metrics_by_gender['female']['equalized_odds'], metrics_by_gender['female']['equal_opportunity']
    ]
}

# Create DataFrame and display
df = pd.DataFrame(data)
csv_file_path = 'late_fusion_metrics_results.csv'
df.to_csv(csv_file_path, index=False)

print(df)

                       Metric  \
0                    Accuracy   
1                   Precision   
2                      Recall   
3                    F1 Score   
4          True Positive Rate   
5         False Positive Rate   
6         False Negative Rate   
7           Predictive Parity   
8               Positive Rate   
9          Demographic Parity   
10  Equalized Odds (FPR, FNR)   
11    Equal Opportunity (TPR)   

                                                 Male  \
0                                             0.93578   
1                                            0.934868   
2                                            0.938095   
3                                            0.932563   
4   [1.0, 1.0, 1.0, 0.7142857142857143, 1.0, 1.0, ...   
5   [0.00980392156862745, 0.010638297872340425, 0....   
6   [0.0, 0.0, 0.0, 0.2857142857142857, 0.0, 0.0, ...   
7   [0.875, 0.9375, 1.0, 0.9090909090909091, 0.882...   
8                                            3.642202   


In [64]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(800)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [65]:
model=multi_modal_NN()

model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("model_final.h5", monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
LRCN_model_training_history = model.fit(          x = X_train,
                                                  y = y_train,
                                                  validation_data=(X_test, y_test),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb,early_stopping_callback])

Epoch 1/80
288/288 [==============================] - 6s 17ms/step - loss: 1.2131 - accuracy: 0.6458 - val_loss: 0.2582 - val_accuracy: 0.9479
Epoch 2/80
288/288 [==============================] - 5s 17ms/step - loss: 1.0488 - accuracy: 0.7439 - val_loss: 0.2190 - val_accuracy: 0.9549
Epoch 3/80
288/288 [==============================] - 5s 16ms/step - loss: 1.0184 - accuracy: 0.7335 - val_loss: 0.2076 - val_accuracy: 0.9583
Epoch 4/80
288/288 [==============================] - 5s 16ms/step - loss: 0.9271 - accuracy: 0.7778 - val_loss: 0.1819 - val_accuracy: 0.9653
Epoch 5/80
288/288 [==============================] - 5s 16ms/step - loss: 0.8443 - accuracy: 0.8151 - val_loss: 0.1856 - val_accuracy: 0.9549
Epoch 6/80
288/288 [==============================] - 4s 16ms/step - loss: 0.8615 - accuracy: 0.7951 - val_loss: 0.1823 - val_accuracy: 0.9583
Epoch 7/80
288/288 [==============================] - 5s 16ms/step - loss: 0.8154 - accuracy: 0.8125 - val_loss: 0.1678 - val_accuracy: 0.9653

In [66]:
#clf = SVC(**trial.params)

TypeError: __init__() got an unexpected keyword argument 'logreg_c'

In [ ]:
#Early Fusion

In [26]:
# dbfile = open('video_data_new2', 'rb')     
# df = pickle.load(dbfile)

# emotion_labels = df['labels']
# genders = df['genders']
# df=df['features']


# dbfile = open('data_mel_new', 'rb')
# df_audio_1=pickle.load(dbfile)
# df_audio = df_audio_1['features']
# labels = df_audio_1['labels']
# genders_audio = df_audio_1['genders']
# paths = df_audio_1['video_files_paths']

In [35]:
# labels = np.array(labels)
# labels.shape

(1440,)

In [16]:
#1 - Flatten
X_train_flattened = np.reshape(X_train, (1008, 20, -1))
X_train_flattened.shape
X_train_aud_flattened = np.reshape(X_train_aud, (1008, -1))
X_train_aud_flattened.shape
# video_data_flat = np.reshape(df, (1440, -1))  # Reshape to (816, 20*48*48) or similar
# audio_data_flat = np.reshape(df_audio, (1440, -1))

(1008, 49152)

In [17]:
X_train_aud_reshape = np.reshape(X_train_aud_flattened, (1008, 1, 49152))

X_train_aud_repeated = np.repeat(X_train_aud_reshape, 20, axis=1)
X_train_aud_repeated = X_train_aud_repeated/255

In [18]:
X_train_aud_repeated.shape

(1008, 20, 49152)

In [19]:
concatenated_train_data = np.concatenate((X_train_flattened, X_train_aud_repeated), axis=-1)

In [20]:
X_val_flattened = np.reshape(X_val, (216, 20, -1))
X_val_flattened.shape
X_val_aud_flattened = np.reshape(X_val_aud, (216, -1))
X_val_aud_flattened.shape

X_val_aud_reshape = np.reshape(X_val_aud_flattened, (216, 1, 49152))

X_val_aud_repeated = np.repeat(X_val_aud_reshape, 20, axis=1)
X_val_aud_repeated = X_val_aud_repeated/255

In [21]:
concatenated_val_data = np.concatenate((X_val_flattened, X_val_aud_repeated), axis=-1)

In [22]:
X_test_flattened = np.reshape(X_test, (216, 20, -1))
X_test_flattened.shape
X_test_aud_flattened = np.reshape(X_test_aud, (216, -1))
X_test_aud_flattened.shape

X_test_aud_reshape = np.reshape(X_test_aud_flattened, (216, 1, 49152))

X_test_aud_repeated = np.repeat(X_test_aud_reshape, 20, axis=1)
X_test_aud_repeated = X_val_aud_repeated/255

In [23]:
concatenated_test_data = np.concatenate((X_test_flattened, X_test_aud_repeated), axis=-1)

In [24]:
def multi_modal_NN(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    
    # Flatten the input
    model.add(Flatten())
    
    model.add(Dense(2096))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))

    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))

    model.add(Dense(8))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    return model

In [25]:
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))
    
    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = self.X[indices]
        y_batch = self.y[indices]
        return X_batch, y_batch
    
    def on_epoch_end(self):
        self.indices = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indices)

# Create data generators
train_generator = DataGenerator(concatenated_train_data, y_train, batch_size=32)
val_generator = DataGenerator(concatenated_val_data, y_val, batch_size=32)

In [26]:
input_shape = (20, 51456)  # Shape of your input data
save_dir = os.path.abspath(DIR)+'/saved_models_early_fusion/'
fold_var=1
model = multi_modal_NN(input_shape)

model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("early_fusion.h5", monitor='val_accuracy', save_best_only=True, mode='max')


early_fusion_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=80,
    shuffle=True,
    callbacks=[checkpoint_cb, early_stopping_callback]
)
model.load_weights(save_dir+"model_"+str(fold_var)+".h5")

# early_fusion_history = model.fit(
#     x = concatenated_train_data,
#     y = y_train,
#     validation_data=(concatenated_val_data, y_val),
#     epochs=80,
#     batch_size=2,
#     shuffle=True,
#     callbacks=[checkpoint_cb, early_stopping_callback]
# )

2024-07-14 03:01:51.438454: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8628142080 exceeds 10% of free system memory.
2024-07-14 03:01:52.293987: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8628142080 exceeds 10% of free system memory.
2024-07-14 03:01:52.779791: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8628142080 exceeds 10% of free system memory.


Epoch 1/80


2024-07-14 03:01:54.871313: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8628142080 exceeds 10% of free system memory.
2024-07-14 03:01:55.169380: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8628142080 exceeds 10% of free system memory.


31/31 [==============================] - 868s 21s/step - loss: 2.4236 - accuracy: 0.1240 - val_loss: 12.0393 - val_accuracy: 0.0990
Epoch 2/80
31/31 [==============================] - 1512s 36s/step - loss: 2.3074 - accuracy: 0.1603 - val_loss: 4.5128 - val_accuracy: 0.1198
Epoch 3/80
31/31 [==============================] - 1711s 42s/step - loss: 2.2011 - accuracy: 0.1552 - val_loss: 2.4045 - val_accuracy: 0.1094
Epoch 4/80
31/31 [==============================] - 466s 15s/step - loss: 2.0756 - accuracy: 0.1583 - val_loss: 2.2634 - val_accuracy: 0.1302
Epoch 5/80
31/31 [==============================] - 1592s 36s/step - loss: 2.0632 - accuracy: 0.1804 - val_loss: 2.1483 - val_accuracy: 0.1302
Epoch 6/80
31/31 [==============================] - 472s 14s/step - loss: 2.0550 - accuracy: 0.1764 - val_loss: 2.1428 - val_accuracy: 0.0990
Epoch 7/80
31/31 [==============================] - 402s 13s/step - loss: 2.0402 - accuracy: 0.2016 - val_loss: 2.1316 - val_accuracy: 0.1198
Epoch 8/80
31

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/var/scratch/mpa326/saved_models_early_fusion/model_1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [28]:
model.save(save_dir + 'early_fusion_model_final_fold_' + str(fold_var) + '.h5')

In [29]:
test_generator = DataGenerator(concatenated_test_data, y_test, batch_size=32)

In [31]:
test_loss, test_accuracy = model.evaluate(concatenated_test_data, y_test)

7/7 [==============================] - 59s 8s/step - loss: 2.6923 - accuracy: 0.2130


In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = concatenated_test_data[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = model.predict(X_gender_test)
    y_pred = np.argmax(y_pred, axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr': fpr,
        'fnr': fnr,
        'precision_value': precision_value,
        'positive_rate': np.mean(y_pred)
    }

# Additional fairness metrics
positive_rates = [metrics_by_gender[gender]['positive_rate'] for gender in metrics_by_gender]
disparate_impact_ratio = min(positive_rates) / max(positive_rates)

print("Gender bias analysis:", metrics_by_gender)
print("Disparate Impact Ratio:", disparate_impact_ratio)

for gender in metrics_by_gender:
    metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['fpr'], metrics_by_gender[gender]['fnr'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']

fpr_values = [metrics_by_gender[gender]['fpr'] for gender in metrics_by_gender]
fnr_values = [metrics_by_gender[gender]['fnr'] for gender in metrics_by_gender]

avg_fpr_values = [np.mean(fpr) for fpr in fpr_values]
avg_fnr_values = [np.mean(fnr) for fnr in fnr_values]

false_positive_equality = max(avg_fpr_values) - min(avg_fpr_values)
false_negative_equality = max(avg_fnr_values) - min(avg_fnr_values)

print("False Positive Equality:", false_positive_equality)
print("False Negative Equality:", false_negative_equality)

4/4 [==============================] - 5s 1s/step
Gender bias analysis: {'female': {'accuracy': 0.21495327102803738, 'precision': 0.057060838033843674, 'recall': 0.19702380952380955, 'f1': 0.08664772727272727, 'tpr': array([0.        , 0.        , 0.64285714, 0.        , 0.        ,
       0.        , 0.93333333, 0.        ]), 'fpr': array([0.        , 0.        , 0.2688172 , 0.        , 0.        ,
       0.        , 0.64130435, 0.        ]), 'fnr': array([1.        , 1.        , 0.35714286, 1.        , 1.        ,
       1.        , 0.06666667, 1.        ]), 'precision_value': array([       nan,        nan, 0.26470588,        nan,        nan,
              nan, 0.19178082,        nan]), 'positive_rate': 4.728971962616822}, 'male': {'accuracy': 0.21100917431192662, 'precision': 0.06435356811862836, 'recall': 0.20535714285714285, 'f1': 0.09233247422680413, 'tpr': array([0.        , 0.        , 0.64285714, 0.        , 0.        ,
       0.        , 1.        , 0.        ]), 'fpr': array

In [35]:
data = {
    'Metric': [
        'Accuracy', 'Precision', 'Recall', 'F1 Score', 'True Positive Rate', 'False Positive Rate',
        'False Negative Rate', 'Predictive Parity', 'Positive Rate', 'Demographic Parity',
        'Equalized Odds (FPR, FNR)', 'Equal Opportunity (TPR)'
    ],
    'Male': [
        metrics_by_gender['male']['accuracy'], metrics_by_gender['male']['precision'],
        metrics_by_gender['male']['recall'], metrics_by_gender['male']['f1'],
        metrics_by_gender['male']['tpr'], metrics_by_gender['male']['fpr'],
        metrics_by_gender['male']['fnr'], metrics_by_gender['male']['precision_value'],
        metrics_by_gender['male']['positive_rate'], metrics_by_gender['male']['demographic_parity'],
        metrics_by_gender['male']['equalized_odds'], metrics_by_gender['male']['equal_opportunity']
    ],
    'Female': [
        metrics_by_gender['female']['accuracy'], metrics_by_gender['female']['precision'],
        metrics_by_gender['female']['recall'], metrics_by_gender['female']['f1'],
        metrics_by_gender['female']['tpr'], metrics_by_gender['female']['fpr'],
        metrics_by_gender['female']['fnr'], metrics_by_gender['female']['precision_value'],
        metrics_by_gender['female']['positive_rate'], metrics_by_gender['female']['demographic_parity'],
        metrics_by_gender['female']['equalized_odds'], metrics_by_gender['female']['equal_opportunity']
    ]
}

# Create DataFrame and display
df = pd.DataFrame(data)
csv_file_path = 'early_fusion_metrics_results14thJul.csv'
df.to_csv(csv_file_path, index=False)

print(df)

                       Metric  \
0                    Accuracy   
1                   Precision   
2                      Recall   
3                    F1 Score   
4          True Positive Rate   
5         False Positive Rate   
6         False Negative Rate   
7           Predictive Parity   
8               Positive Rate   
9          Demographic Parity   
10  Equalized Odds (FPR, FNR)   
11    Equal Opportunity (TPR)   

                                                 Male  \
0                                            0.211009   
1                                            0.064354   
2                                            0.205357   
3                                            0.092332   
4   [0.0, 0.0, 0.6428571428571429, 0.0, 0.0, 0.0, ...   
5   [0.0, 0.0, 0.17894736842105263, 0.0, 0.0, 0.0,...   
6   [1.0, 1.0, 0.35714285714285715, 1.0, 1.0, 1.0,...   
7   [nan, nan, 0.34615384615384615, nan, nan, nan,...   
8                                            5.045872   


In [121]:
# def create__LRCN_with_STN():
#     concatenated_features = tf.keras.layers.Input(shape=(95232,))
#     x = tf.keras.layers.Reshape((20, 4761))(concatenated_features)
    
#     lstm = Bidirectional(LSTM(200, activation='tanh', input_shape=(20, 4761), dropout=0.3))(x)
    
#     out = Dense(len(emotions_en), activation='softmax')(lstm)

#     model = tf.keras.models.Model(inputs=concatenated_features, outputs=out)
#     return model

In [36]:
# combined_labels = [(e, g) for e, g in zip(labels, genders)]
# X_train, X_temp, y_train, y_temp, gender_train, gender_temp = train_test_split(
#     concatenated_data, labels, genders, test_size=0.3, stratify=combined_labels, random_state=42
# )

# combined_temp_labels = [(e, g) for e, g in zip(y_temp, gender_temp)]

# # Split the temporary set into validation and test sets
# X_val, X_test, y_val, y_test, gender_val, gender_test = train_test_split(
#     X_temp, y_temp, gender_temp, test_size=0.5, stratify=combined_temp_labels, random_state=42
# )

# print("Train set shape:", X_train.shape)
# print("Validation set shape:", X_val.shape)
# print("Test set shape:", X_test.shape)

# print("Label Train set shape:", y_train.shape)
# print("Label Validation set shape:", y_val.shape)
# print("Label Test set shape:", y_test.shape)

# print("GenderTrain set shape:", gender_train)
# print("Gender Validation set shape:", gender_val.shape)
# print("Gender Test set shape:", gender_test.shape)

Train set shape: (1008, 95232)
Validation set shape: (216, 95232)
Test set shape: (216, 95232)
Label Train set shape: (1008,)
Label Validation set shape: (216,)
Label Test set shape: (216,)
GenderTrain set shape: ['female' 'male' 'female' ... 'female' 'male' 'male']
Gender Validation set shape: (216,)
Gender Test set shape: (216,)


In [42]:
import gc

import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

import optuna

In [43]:
study = optuna.create_study(study_name="tps-feb-2022", direction="maximize")
study.optimize(objective, n_trials=30)

[I 2024-06-16 12:52:14,944] A new study created in memory with name: tps-feb-2022
[I 2024-06-16 12:52:51,752] Trial 0 finished with value: 0.6481481481481481 and parameters: {'logreg_c': 5705708424.191827}. Best is trial 0 with value: 0.6481481481481481.
[I 2024-06-16 12:53:35,833] Trial 1 finished with value: 0.5972222222222222 and parameters: {'logreg_c': 5.1299049881000455e-09}. Best is trial 0 with value: 0.6481481481481481.
[I 2024-06-16 12:54:10,721] Trial 2 finished with value: 0.6481481481481481 and parameters: {'logreg_c': 1445315.6881947566}. Best is trial 0 with value: 0.6481481481481481.
[I 2024-06-16 12:54:57,640] Trial 3 finished with value: 0.5231481481481481 and parameters: {'logreg_c': 2.471575396723528e-09}. Best is trial 0 with value: 0.6481481481481481.
[I 2024-06-16 12:55:41,436] Trial 4 finished with value: 0.6759259259259259 and parameters: {'logreg_c': 2.527992265558401e-06}. Best is trial 4 with value: 0.6759259259259259.
[I 2024-06-16 12:56:16,799] Trial 5 fin

In [44]:
trial = study.best_trial
print("  Best accuracy: ", study.best_value)
print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Best accuracy:  0.7037037037037037
  Value: 0.7037037037037037
  Params: 
    logreg_c: 1.233381092916111e-07


In [45]:
from sklearn.linear_model import LogisticRegression
best_params = study.best_params
logreg_c = best_params["logreg_c"]
X_combined = np.vstack((X_train, X_val))
y_combined = np.hstack((y_train, y_val))
best_clf = LogisticRegression(C=logreg_c)
best_clf.fit(X_combined, y_combined)

LogisticRegression(C=1.233381092916111e-07)

In [46]:
best_preds = best_clf.predict(X_test)

In [48]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
overall_metrics = {
    'accuracy': accuracy_score(y_test, best_preds),
    'precision': precision_score(y_test, best_preds, average='macro'),
    'recall': recall_score(y_test, best_preds, average='macro'),
    'f1_score': f1_score(y_test, best_preds, average='macro')
}

print("Test set evaluation:", overall_metrics)

Test set evaluation: {'accuracy': 0.7037037037037037, 'precision': 0.7026678783971783, 'recall': 0.7007491789819376, 'f1_score': 0.6976624063049259}


In [49]:
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = X_test[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = best_clf.predict(X_gender_test)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr': fpr,
        'fnr': fnr,
        'precision_value': precision_value,
        'positive_rate': np.mean(y_pred)
    }

# Additional fairness metrics
positive_rates = [metrics_by_gender[gender]['positive_rate'] for gender in metrics_by_gender]
disparate_impact_ratio = min(positive_rates) / max(positive_rates)

print("Gender bias analysis:", metrics_by_gender)
print("Disparate Impact Ratio:", disparate_impact_ratio)

for gender in metrics_by_gender:
    metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['fpr'], metrics_by_gender[gender]['fnr'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']

fpr_values = [metrics_by_gender[gender]['fpr'] for gender in metrics_by_gender]
fnr_values = [metrics_by_gender[gender]['fnr'] for gender in metrics_by_gender]

avg_fpr_values = [np.mean(fpr) for fpr in fpr_values]
avg_fnr_values = [np.mean(fnr) for fnr in fnr_values]

false_positive_equality = max(avg_fpr_values) - min(avg_fpr_values)
false_negative_equality = max(avg_fnr_values) - min(avg_fnr_values)

print("False Positive Equality:", false_positive_equality)
print("False Negative Equality:", false_negative_equality)

Gender bias analysis: {'female': {'accuracy': 0.7102803738317757, 'precision': 0.7141739388062918, 'recall': 0.7029761904761904, 'f1': 0.70165816334615, 'tpr': array([0.57142857, 0.85714286, 0.78571429, 0.8       , 0.46666667,
       0.85714286, 0.78571429, 0.5       ]), 'fpr': array([0.02      , 0.03225806, 0.01075269, 0.08695652, 0.0326087 ,
       0.05376344, 0.03225806, 0.06451613]), 'fnr': array([0.42857143, 0.14285714, 0.21428571, 0.2       , 0.53333333,
       0.14285714, 0.21428571, 0.5       ]), 'precision_value': array([0.66666667, 0.8       , 0.91666667, 0.6       , 0.7       ,
       0.70588235, 0.78571429, 0.53846154]), 'positive_rate': 3.7289719626168223}, 'male': {'accuracy': 0.6972477064220184, 'precision': 0.6980881691408007, 'recall': 0.6952380952380952, 'f1': 0.6835446207593774, 'tpr': array([0.71428571, 0.93333333, 0.66666667, 0.42857143, 0.6       ,
       0.64285714, 0.64285714, 0.93333333]), 'fpr': array([0.03921569, 0.05319149, 0.0212766 , 0.02105263, 0.04255319

In [50]:
data = {
    'Metric': [
        'Accuracy', 'Precision', 'Recall', 'F1 Score', 'True Positive Rate', 'False Positive Rate',
        'False Negative Rate', 'Predictive Parity', 'Positive Rate', 'Demographic Parity',
        'Equalized Odds (FPR, FNR)', 'Equal Opportunity (TPR)'
    ],
    'Male': [
        metrics_by_gender['male']['accuracy'], metrics_by_gender['male']['precision'],
        metrics_by_gender['male']['recall'], metrics_by_gender['male']['f1'],
        metrics_by_gender['male']['tpr'], metrics_by_gender['male']['fpr'],
        metrics_by_gender['male']['fnr'], metrics_by_gender['male']['precision_value'],
        metrics_by_gender['male']['positive_rate'], metrics_by_gender['male']['demographic_parity'],
        metrics_by_gender['male']['equalized_odds'], metrics_by_gender['male']['equal_opportunity']
    ],
    'Female': [
        metrics_by_gender['female']['accuracy'], metrics_by_gender['female']['precision'],
        metrics_by_gender['female']['recall'], metrics_by_gender['female']['f1'],
        metrics_by_gender['female']['tpr'], metrics_by_gender['female']['fpr'],
        metrics_by_gender['female']['fnr'], metrics_by_gender['female']['precision_value'],
        metrics_by_gender['female']['positive_rate'], metrics_by_gender['female']['demographic_parity'],
        metrics_by_gender['female']['equalized_odds'], metrics_by_gender['female']['equal_opportunity']
    ]
}

# Create DataFrame and display
df = pd.DataFrame(data)
csv_file_path = 'early_fusion_metrics_results.csv'
df.to_csv(csv_file_path, index=False)

print(df)

                       Metric  \
0                    Accuracy   
1                   Precision   
2                      Recall   
3                    F1 Score   
4          True Positive Rate   
5         False Positive Rate   
6         False Negative Rate   
7           Predictive Parity   
8               Positive Rate   
9          Demographic Parity   
10  Equalized Odds (FPR, FNR)   
11    Equal Opportunity (TPR)   

                                                 Male  \
0                                            0.697248   
1                                            0.698088   
2                                            0.695238   
3                                            0.683545   
4   [0.7142857142857143, 0.9333333333333333, 0.666...   
5   [0.0392156862745098, 0.05319148936170213, 0.02...   
6   [0.2857142857142857, 0.06666666666666667, 0.33...   
7   [0.5555555555555556, 0.7368421052631579, 0.833...   
8                                            3.788991   


In [ ]:
#Cross-Modal Fusion

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Flatten, Add, Multiply, Input
from tensorflow.keras.models import Model
import os

In [13]:
class CrossAttention(Layer):
    def __init__(self, num_heads, key_dim):
        super(CrossAttention, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.query_dense = Dense(key_dim * num_heads, use_bias=False)
        self.key_dense = Dense(key_dim * num_heads, use_bias=False)
        self.value_dense = Dense(key_dim * num_heads, use_bias=False)
        self.combine_heads = Dense(key_dim * num_heads)

    def call(self, query, context):
        query = self.query_dense(query)
        key = self.key_dense(context)
        value = self.value_dense(context)
        
        query = tf.reshape(query, (-1, query.shape[1], self.num_heads, self.key_dim))
        key = tf.reshape(key, (-1, key.shape[1], self.num_heads, self.key_dim))
        value = tf.reshape(value, (-1, value.shape[1], self.num_heads, self.key_dim))

        attention = tf.einsum('bijh,bjkh->bikh', query, key) / (self.key_dim ** 0.5)
        attention_weights = tf.nn.softmax(attention, axis=-1)
        out = tf.einsum('bikh,bjkh->bijh', attention_weights, value)
        out = tf.reshape(out, (-1, query.shape[1], self.num_heads * self.key_dim))
        
        return self.combine_heads(out)

In [14]:
def create_crossmodal_model(visual_model, audio_model, num_classes):
    #visual_input = visual_model.input
    visual_features = visual_model.input

    #audio_input = Input(shape=(128, 128, 3))
    #audio_input = audio_model.input
    audio_features = audio_model.input
    #audio_features = audio_model(audio_input)

    # Self-Attention for visual and audio features
    visual_attention = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=64)(visual_features, visual_features)
    audio_attention = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=64)(audio_features, audio_features)

    # Cross-Attention from visual to audio and vice versa
    visual_to_audio_attention = CrossAttention(num_heads=8, key_dim=64)(visual_attention, audio_attention)
    audio_to_visual_attention = CrossAttention(num_heads=8, key_dim=64)(audio_attention, visual_attention)

    # Combine the features
    combined_features = Add()([visual_attention, visual_to_audio_attention])
    combined_features = Multiply()([combined_features, audio_to_visual_attention])

    # Fully connected layers
    combined_features = Flatten()(combined_features)
    output = Dense(num_classes, activation='softmax')(combined_features)

    return Model(inputs=[visual_input, audio_input], outputs=output)

save_dir = os.path.abspath(DIR)+'/saved_models_lrcn_stn/'
final_model_video = create__LRCN_with_STN()
final_model_video.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model_video.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

final_model_video = Model(inputs=final_model_video.input, outputs=final_model_video.get_layer('time_distributed_2').output)

save_dir_aud = os.path.abspath(DIR)+'/saved_model_feature_extraction_audio/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir_aud +"model_1.h5")
model_audio = Model(inputs=model_audio.input, outputs=model_audio.get_layer('conv2d_5').output)
num_classes = 8

combined_model = create_crossmodal_model(final_model_video, model_audio, num_classes)

    

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


ValueError: No such layer: conv2d_5. Existing layers are: ['conv2d_7', 'batch_normalization', 'activation', 'max_pooling2d_3', 'conv2d_8', 'batch_normalization_1', 'activation_1', 'max_pooling2d_4', 'conv2d_9', 'batch_normalization_2', 'activation_2', 'conv2d_10', 'batch_normalization_3', 'activation_3', 'conv2d_11', 'batch_normalization_4', 'activation_4', 'max_pooling2d_5', 'flatten_2', 'dense_6', 'batch_normalization_5', 'activation_5', 'dropout_2', 'dense_7', 'batch_normalization_6', 'activation_6', 'dropout_3', 'dense_8', 'batch_normalization_7', 'activation_7'].

In [65]:
save_dir = os.path.abspath(DIR)+'/saved_models_lrcn_stn/'
final_model_video = create__LRCN_with_STN()
final_model_video.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model_video.load_weights(save_dir + "model_1.h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [66]:
final_model_video.input

<KerasTensor: shape=(None, 20, 48, 48, 1) dtype=float32 (created by layer 'input_6')>

In [67]:
final_model_video.output

<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'dense_23')>

In [71]:
model_audio.input


<KerasTensor: shape=(None, 128, 128, 3) dtype=float32 (created by layer 'conv2d_30_input')>

In [70]:
model_audio.output

<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'activation_31')>

In [ ]:
#Intermediate fusion

In [20]:
from tensorflow.keras.models import load_model
indice_fold = 1
save_dir = os.path.abspath(DIR)+'/saved_model_feature_extraction_audio/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(indice_fold)+".h5")

In [21]:
save_dir = os.path.abspath(DIR)+'/saved_models_lrcn_stn/'

model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(indice_fold)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [24]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('time_distributed_5').output)

In [25]:
features_video=features_video_model.predict(X_train)

32/32 [==============================] - 7s 191ms/step


In [26]:
features_video.shape

(1008, 20, 1024)

In [28]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('conv2d_16').output)

In [31]:
features_audio = features_audio_model.predict(X_train_aud)

32/32 [==============================] - 12s 293ms/step


In [32]:
features_audio.shape

(1008, 32, 32, 256)

In [50]:
features_video_val = features_video_model.predict(X_val)
features_video_test = features_video_model.predict(X_test)

features_audio_val = features_audio_model.predict(X_val_aud)
features_audio_test = features_audio_model.predict(X_test_aud)

7/7 [==============================] - 2s 275ms/step


In [52]:
features_audio_test.shape

(216, 32, 32, 256)

In [55]:
features_audio_flattened = np.reshape(features_audio, (1008, -1))
features_audio_flattened_val = np.reshape(features_audio_val, (216, -1))
features_audio_flattened_test = np.reshape(features_audio_test, (216, -1))
features_audio_flattened_val.shape



(216, 262144)

In [56]:
from sklearn.decomposition import PCA

lda = PCA(n_components=1000)
features_audio_flattened = lda.fit_transform(features_audio_flattened)
features_audio_flattened_val = lda.transform(features_audio_flattened_val)
features_audio_flattened_test = lda.transform(features_audio_flattened_test)

# X_test = lda.transform(X_test)
# X_val = lda.transform(X_val)


In [57]:
features_audio_flattened_val.shape

(216, 1000)

In [58]:
features_audio_flattened_val

array([[-5.10063721e+02,  5.79397827e+02, -4.13256500e+02, ...,
        -2.42818222e+01,  5.88011932e+00,  1.06233454e+00],
       [ 1.31789734e+03, -2.95173248e+02, -5.66947510e+02, ...,
        -6.98942757e+00,  3.30174279e+00,  1.11193764e+00],
       [-5.96122986e+02, -1.26955925e+02, -3.18186874e+01, ...,
         3.06036329e+00,  4.48986721e+00, -5.27141380e+00],
       ...,
       [-8.35318115e+02,  2.15608459e+02, -3.29985443e+02, ...,
        -2.29646778e+00,  5.72856855e+00,  1.25495796e+01],
       [-9.42429993e+02, -7.86249771e+01, -4.49961273e+02, ...,
         6.77876472e-01,  1.25635633e+01, -1.15308666e+01],
       [ 8.87471436e+02,  4.45727921e+01, -5.61877686e+02, ...,
         1.15943079e+01,  2.06615200e+01, -6.21164131e+00]], dtype=float32)

In [61]:
features_audio_reshape = np.reshape(features_audio_flattened, (1008, 1, 1000))
features_audio_repeated = np.repeat(features_audio_reshape, 20, axis=1)

features_audio_reshape_val = np.reshape(features_audio_flattened_val, (216, 1, 1000))
features_audio_repeated_val = np.repeat(features_audio_reshape_val, 20, axis=1)

features_audio_reshape_test = np.reshape(features_audio_flattened_test, (216, 1, 1000))
features_audio_repeated_test = np.repeat(features_audio_reshape_test, 20, axis=1)
# X_val_aud_repeated = X_val_aud_repeated/255
features_audio_repeated_test.shape

(216, 20, 1000)

In [62]:
concatenated_train_data = np.concatenate((features_video, features_audio_repeated), axis=-1)
concatenated_val_data = np.concatenate((features_video_val, features_audio_repeated_val), axis=-1)
concatenated_test_data = np.concatenate((features_video_test, features_audio_repeated_test), axis=-1)

In [63]:
concatenated_train_data.shape

(1008, 20, 2024)

In [68]:
num_classes = len(np.unique(y_train))
print(num_classes)
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

8


In [69]:
def create_model(trial):
    model = Sequential()

    # Suggest the number of LSTM units
    lstm_units = trial.suggest_int("lstm_units", 32, 256)
    model.add(LSTM(lstm_units, input_shape=(concatenated_train_data.shape[1], concatenated_train_data.shape[2]), return_sequences=False))

    # Suggest a dropout rate
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    model.add(Dropout(dropout_rate))

    # Add a dense layer with softmax activation for multi-class classification
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [75]:
# def objective(trial):
#     logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
#     X_combined = np.vstack((concatenated_train_data, concatenated_val_data))
#     y_combined = np.hstack((y_train, y_val))
    
#     # Initialize the logistic regression model with the suggested C value
#     clf = linear_model.LogisticRegression(C=logreg_c)
    
#     # Fit the model on the combined training and validation data
#     clf.fit(X_combined, y_combined)
#     preds = clf.predict(concatenated_test_data)
#     score = accuracy_score(y_test, preds)
#     gc.collect()
#     return score

def objective(trial):
    model = create_model(trial)
    
    # Fit the model on the training data
    model.fit(concatenated_train_data, y_train, epochs=80, batch_size=16, validation_data=(concatenated_val_data, y_val), verbose=0)
    
    # Evaluate the model on the test data
    preds = model.predict(concatenated_test_data)
    preds = np.argmax(preds, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)
    score = accuracy_score(y_test_labels, preds)
    
    # Clean up memory
    gc.collect()
    
    return score

In [76]:
import optuna
from sklearn.metrics import accuracy_score
import gc
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2024-07-14 22:58:56,779] A new study created in memory with name: no-name-bcc86627-bddb-4f8c-a3bb-24cac29d3ec9


7/7 [==============================] - 1s 20ms/step


[I 2024-07-14 23:01:57,290] Trial 0 finished with value: 0.1388888888888889 and parameters: {'lstm_units': 241, 'dropout_rate': 0.3827106750177438}. Best is trial 0 with value: 0.1388888888888889.


7/7 [==============================] - 0s 10ms/step


[I 2024-07-14 23:03:19,324] Trial 1 finished with value: 0.11574074074074074 and parameters: {'lstm_units': 74, 'dropout_rate': 0.19331554189753747}. Best is trial 0 with value: 0.1388888888888889.


...

7/7 [==============================] - 1s 18ms/step


[I 2024-07-14 23:19:59,977] Trial 10 finished with value: 0.13425925925925927 and parameters: {'lstm_units': 172, 'dropout_rate': 0.4909920772539739}. Best is trial 4 with value: 0.16666666666666666.


7/7 [==============================] - 0s 17ms/step


[I 2024-07-14 23:22:05,694] Trial 11 finished with value: 0.14814814814814814 and parameters: {'lstm_units': 154, 'dropout_rate': 0.22427138283832374}. Best is trial 4 with value: 0.16666666666666666.
[W 2024-07-14 23:23:42,214] Trial 12 failed with parameters: {'lstm_units': 126, 'dropout_rate': 0.42887831517757125} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/var/scratch/mpa326/miniconda3/envs/tf/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_21469/3251019886.py", line 20, in objective
    model.fit(concatenated_train_data, y_train, epochs=80, batch_size=16, validation_data=(concatenated_val_data, y_val), verbose=0)
  File "/var/scratch/mpa326/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/var/scratch/mpa326/miniconda3/envs/tf/lib/python3.9/si

KeyboardInterrupt: 

In [77]:
features_video_flattened = np.reshape(features_video, (1008, -1))
features_video_flattened_val = np.reshape(features_video_val, (216, -1))
features_video_flattened_test = np.reshape(features_video_test, (216, -1))

In [78]:
concatenated_train_data = np.concatenate((features_video_flattened, features_audio_flattened), axis=-1)
concatenated_val_data = np.concatenate((features_video_flattened_val, features_audio_flattened_val), axis=-1)
concatenated_test_data = np.concatenate((features_video_flattened_test, features_audio_flattened_test), axis=-1)

In [79]:
def objective(trial):
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    X_combined = np.vstack((concatenated_train_data, concatenated_val_data))
    y_combined = np.hstack((y_train, y_val))
    
    # Initialize the logistic regression model with the suggested C value
    clf = linear_model.LogisticRegression(C=logreg_c)
    
    # Fit the model on the combined training and validation data
    clf.fit(X_combined, y_combined)
    preds = clf.predict(concatenated_test_data)
    score = accuracy_score(y_test, preds)
    gc.collect()
    return score

In [87]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2024-07-14 23:25:33,962] A new study created in memory with name: no-name-acb75a6c-6b7f-4b2d-9401-024ff6fd76b6
[I 2024-07-14 23:26:03,166] Trial 0 finished with value: 0.1388888888888889 and parameters: {'logreg_c': 7.476308017052857e-05}. Best is trial 0 with value: 0.1388888888888889.
[I 2024-07-14 23:26:19,757] Trial 1 finished with value: 0.14351851851851852 and parameters: {'logreg_c': 13548729.740835818}. Best is trial 1 with value: 0.14351851851851852.
[I 2024-07-14 23:26:35,464] Trial 2 finished with value: 0.14351851851851852 and parameters: {'logreg_c': 412710128.95377797}. Best is trial 1 with value: 0.14351851851851852.
[I 2024-07-14 23:27:04,413] Trial 3 finished with value: 0.13425925925925927 and parameters: {'logreg_c': 0.0004987116260962908}. Best is trial 1 with value: 0.14351851851851852.
[I 2024-07-14 23:27:33,259] Trial 4 finished with value: 0.12962962962962962 and parameters: {'logreg_c': 3.171930631946704e-05}. Best is trial 1 with value: 0.14351851851851852.